In [2]:
# 멀티스레딩

from threading import Thread

def my_function():
    print("printing from thread")
    
threads = [Thread(target=my_function) for _ in range(10)]

for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()

printing from threadprinting from thread

printing from thread
printing from thread
printing from thread
printing from thread
printing from thread
printing from thread
printing from thread
printing from thread


In [9]:
from threading import Lock

thread_visits = 0
thread_visits_lock = Lock()

def visit_counter():
    global thread_visits
    for i in range(100_00):
        with thread_visits_lock:
            thread_visits +=1
                
thread_count = 100

threads = [
    Thread(target=visit_counter)
    for _ in range(thread_count)
]

for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()
    
print(f"{thread_count=}, {thread_visits=}")

thread_count=100, thread_visits=1000000


In [10]:
# thread 실행시 Lock를 사용해서 한순간에 한 스레드만 단일 코드 블록에 접근하는것을 보장한다.

# thread는 작업 위임이나 백그라운드 프로세싱에 유리함
# 외부 서비스에 대한 여러 네트워크 요청을 수행하는 것 (ex google maps API 연동)


In [12]:
import requests

response = requests.get("https://api.vatcomply.com/rates?base=USD")
response.json()

{'date': '2023-03-17',
 'base': 'USD',
 'rates': {'EUR': 0.9413536665725313,
  'USD': 1.0,
  'JPY': 132.32608491010072,
  'BGN': 1.8410995010825566,
  'CZK': 22.543537607078978,
  'DKK': 7.008378047632496,
  'GBP': 0.8235714958109761,
  'HUF': 372.7101572060623,
  'PLN': 4.428598324390474,
  'RON': 4.6322131224701115,
  'SEK': 10.542690388779064,
  'CHF': 0.9279864445072014,
  'ISK': 140.54410241927894,
  'NOK': 10.751953308858138,
  'TRY': 19.00207097806646,
  'AUD': 1.494304810317236,
  'BRL': 5.259531205874047,
  'CAD': 1.3728701873293796,
  'CNY': 6.882895603878377,
  'HKD': 7.849665819448366,
  'IDR': 15374.206909535913,
  'ILS': 3.6766450155323356,
  'INR': 82.6386143274028,
  'KRW': 1308.3309799491667,
  'MXN': 18.810976183752235,
  'MYR': 4.485456085851454,
  'NZD': 1.6016191283065047,
  'PHP': 54.70959239386237,
  'SGD': 1.342558599265744,
  'THB': 34.12030499858797,
  'ZAR': 18.35526687376447}}

In [16]:
import time

SYMBOLS = ("USD", "EUR", "PLN", "NOK", "CZK")
BASES = ("USD", "EUR", "PLN", "NOK", "CZK")

def fetch_rates(base):
    response = requests.get(f"https://api.vatcomply.com/rates?base={base}")
    response.raise_for_status()
    rates = response.json()["rates"]
    
    # 동일 화폐는 1:1로 환전한다.
    rates[base] = 1.0
    
    rates_line = ", ".join(
        [f"{rates[symbol]:7.03} {symbol}" for symbol in SYMBOLS]
    )
    print(f"1 {base} = {rates_line}")
          
def main():
    for base in BASES:
          fetch_rates(base)
          
started = time.time()
main()
elapsed = time.time() - started
          
print()
print("time elased : {:.2f}s".format(elapsed))

1 USD =     1.0 USD,   0.941 EUR,    4.43 PLN,    10.8 NOK,    22.5 CZK
1 EUR =    1.06 USD,     1.0 EUR,     4.7 PLN,    11.4 NOK,    23.9 CZK
1 PLN =   0.226 USD,   0.213 EUR,     1.0 PLN,    2.43 NOK,    5.09 CZK
1 NOK =   0.093 USD,  0.0876 EUR,   0.412 PLN,     1.0 NOK,     2.1 CZK
1 CZK =  0.0444 USD,  0.0418 EUR,   0.196 PLN,   0.477 NOK,     1.0 CZK

time elased : 3.57s


In [19]:
# Thread를 활용하여 처리해보자 (통신하는 서비스가 여러 요청을 동시에 처리하고 있다면 분명 성능이 개선됨을 볼수 있다.)

def main():
    threads = []
    
    for base in BASES:
        thread = Thread(target=fetch_rates, args=[base])
        thread.start()
        threads.append(thread)
        
    while threads:
        threads.pop().join()
        
started = time.time()
main()
elapsed = time.time() - started
          
print()
print("time elased : {:.2f}s".format(elapsed))

1 CZK =  0.0444 USD,  0.0418 EUR,   0.196 PLN,   0.477 NOK,     1.0 CZK
1 EUR =    1.06 USD,     1.0 EUR,     4.7 PLN,    11.4 NOK,    23.9 CZK
1 NOK =   0.093 USD,  0.0876 EUR,   0.412 PLN,     1.0 NOK,     2.1 CZK
1 USD =     1.0 USD,   0.941 EUR,    4.43 PLN,    10.8 NOK,    22.5 CZK
1 PLN =   0.226 USD,   0.213 EUR,     1.0 PLN,    2.43 NOK,    5.09 CZK

time elased : 0.78s


In [ ]:
# 스레드 풀 이용하기

from queue import Queue
from queue import Empty

THREAD_POOL_SIZE = 4

def worker(work_queue):
    while not work_queue.empty():
        try:
            item = work_queue.get_nowait()
        except Empty:
            break
        else:
            fetch_rates(item)
            work_queue.task_done()

def main():
    work_queue = Queue()
    
    for base in BASES:
        work_queue.put(base)
        
    threads = [
        Thread(target=worker, args=(work_queue,))
        for _ in range(THREAD_POOL_SIZE)
    ]
    
    for thread in threads:
        thread.start()
        
    work_queue.join()
    
    while threads:
        threads.pop().join()
        
started = time.time()
main()
elapsed = time.time() - started
          
print()
print("time elased : {:.2f}s".format(elapsed))

11 ['1-1 ae_txn_summary.txt', '1-2 ae_txn_sql_summary_top.txt', '1-3 ae_txn_sql_summary_1day.txt', '2-1 ae_txn_summary.txt', '2-2 ae_txn_sql_summary_top.txt', '2-3 ae_txn_sql_summary_1day.txt']
